In [1]:
import numpy as np
from numpy import shape, matrix, log, exp, zeros,random,dot,multiply
from mnist import readWithoutBias
from math import sqrt

In [9]:
data_train, label_train = readWithoutBias(dataset="training")
data_test, label_test = readWithoutBias(dataset="testing")
label_train = matrix(label_train)
label_test  = matrix(label_test)
data_train  = matrix(data_train).T
data_test   = matrix(data_test).T
data_train  = zscore(data_train)
data_test   = zscore(data_test)

/Users/apoorve/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: RuntimeWarning: invalid value encountered in divide
  from IPython.kernel.zmq import kernelapp as app


##Code with full input (refer Checkpoint 1 for 1 input)

In [10]:
n_input = shape(data_train)[0] # excluding bias term
n_hidden = 100                  # excluding bias term
n_output = 10
epsilon = 0.7
random.seed(0)
theta1 = matrix(random.randn(n_hidden,n_input+1))/sqrt(n_input)
theta2 = matrix(random.randn(n_output,n_hidden+1))/sqrt(n_hidden)
#shape(theta1),shape(theta2)
#((20, 785), (10, 21))

### Training Code

In [11]:
X = data_train
y = label_train

In [15]:
X = data_train
y = label_train
X_test = data_test
y_test = label_test
alpha = 0.00001
random.seed(0)
count = 0
while(True):
    i = random.randint(0,60000)
    gradTheta1, gradTheta2 = backPropGradient(X[:,i], y[i], theta1, theta2) #just one example passed
    theta1 = theta1 - alpha*gradTheta1
    theta2 = theta2 - alpha*gradTheta2
    count += 1
    if count == 100:
        count = 0
        #print errorAll(matrix(data_train), label_train, theta1, theta2)
        print accuracy(X, y, theta1, theta2),accuracy(X_test, y_test, theta1, theta2) # get error on complete data

0.883416666667 0.8857
0.8834 0.8857
0.883366666667 0.8856
0.883366666667 0.8856
0.88335 0.8856
0.8834 0.8855
0.883383333333 0.8856
0.883416666667 0.8856
0.883416666667 0.8856
0.883416666667 0.8856
0.8834 0.8856
0.883416666667 0.8856
0.883416666667 0.8856
0.883416666667 0.8856
0.8834 0.8856
0.883433333333 0.8856
0.883466666667 0.8856
0.883466666667 0.8857
0.88345 0.8856
0.88345 0.8856
0.88345 0.8856
0.883433333333 0.8856
0.883433333333 0.8856
0.883433333333 0.8856
0.883366666667 0.8856
0.88335 0.8856
0.883383333333 0.8856
0.88335 0.8856
0.883383333333 0.8856
0.883366666667 0.8856
0.883416666667 0.8857
0.883416666667 0.8856
0.8834 0.8856
0.883416666667 0.8856
0.88345 0.8856
0.88345 0.8856


KeyboardInterrupt: 

In [45]:
shape(numGrad1-gradTheta1),shape(numGrad2),shape(gradTheta2), shape(gradTheta1), type(t)

((20, 785), (10, 21), (10, 21), (20, 785), numpy.matrixlib.defmatrix.matrix)

### Checking backpropgradient vs numerical gradient

In [12]:
gradTheta1, gradTheta2 = backPropGradient(X[:,0],y[0],theta1,theta2)
numGrad1,numGrad2      = numericalGradient(X[:,0],y[0],theta1,theta2)
print np.sum(np.subtract(numGrad2,gradTheta2))
print np.sum(np.subtract(numGrad1,gradTheta1))

-6.50634039701e-10
-3.4210120117e-08


### Helper Functions

In [3]:
def zscore(z):
    return np.nan_to_num((z - z.mean(axis=1))/z.std(axis=1))

In [4]:
def accuracy(X, y, theta1, theta2):
    n_examples = shape(X)[1]
    a1 = np.vstack((np.ones((1,n_examples)),X))
    z2 = dot(theta1,a1)
    a2 = np.vstack((np.ones((1,shape(z2)[1])),sigmoid(z2)))
    z3 = dot(theta2,a2)
    a3 = sigmoid(z3)
    
    pred = np.argmax(a3,axis=0)
    return np.sum(np.equal(pred,y.T))/float(len(y))
    

In [5]:
def backPropGradient(X, y, theta1, theta2):
    n_examples = shape(X)[1]
    a1 = np.vstack((np.ones((1,n_examples)),X))
    z2 = dot(theta1,a1)
    a2 = np.vstack((np.ones((1,shape(z2)[1])),sigmoid(z2)))
    z3 = dot(theta2,a2)
    a3 = sigmoid(z3)
    
    t = np.zeros_like(a3)
    for i in range(len(y)):
        t[y[i],i] = 1
    
    delta3 = a3-t
    delta2 = dot(theta2.T,delta3)[1:]
    delta2 = multiply(delta2,sigmoidGradient(z2))

    gradTheta2 = dot(delta3,a2.T)
    gradTheta1 = dot(delta2,a1.T)
    return gradTheta1, gradTheta2

In [6]:
def numericalGradient(X, y, theta1, theta2):
    epsilon = 10**-5
    numGrad1 = np.zeros_like(theta1)
    numGrad2 = np.zeros_like(theta2)
    for i in range(shape(theta1)[0]):
        for j in range(shape(theta1)[1]):
            theta1_pos = np.copy(theta1)
            theta1_neg = np.copy(theta1)
            theta1_pos[i,j] += epsilon
            theta1_neg[i,j] -= epsilon
            numGrad1[i,j]    = (errorFn(X, y, theta1_pos, theta2) - errorFn(X, y, theta1_neg, theta2))/2/epsilon
    for i in range(shape(theta2)[0]):
        for j in range(shape(theta2)[1]):
            theta2_pos = np.copy(theta2)
            theta2_neg = np.copy(theta2)
            theta2_pos[i,j] += epsilon
            theta2_neg[i,j] -= epsilon
            numGrad2[i,j]    = (errorFn(X, y, theta1, theta2_pos) - errorFn(X, y, theta1, theta2_neg))/2/epsilon
    return numGrad1,numGrad2

In [7]:
def errorFn(X, y, theta1, theta2):
    n_examples = shape(X)[1]
    a1 = np.vstack((np.ones((1,n_examples)),X))
    z2 = dot(theta1,a1)
    a2 = np.vstack((np.ones((1,shape(z2)[1])),sigmoid(z2)))
    z3 = dot(theta2,a2)
    a3 = sigmoid(z3)

    t = np.zeros_like(a3)
    for i in range(len(y)):
        t[y[i],i] = 1

    error = np.sum(multiply(t,log(a3)) + multiply((1-t),log(1-a3)))
    error = -error/n_examples
    return error

In [8]:
def sigmoid(z):
    return 1/(1+exp(-z))
def sigmoidGradient(z):
    s = sigmoid(z)
    return multiply(s,1-s)

In [9]:
shape(pred),shape(y_test)

NameError: name 'pred' is not defined

In [ ]:
pred = accuracy(X_test,y_test,theta1,theta2)

In [ ]:
shape(yu),type(yu),shape(pu),type(pu)

In [ ]:
yu =np.array(y_test).flatten()
pu = np.array(pred).flatten()

In [ ]:
import seaborn as sns
from seaborn import plt
from sklearn import metrics


In [ ]:
label_names = [0,1,2,3,4,5,6,7,8,9]
sns.set_style("white")
sns.axes_style("dark")
plt.figure(figsize=(5, 5))
plt.imshow(metrics.confusion_matrix(yu, pu), interpolation='nearest', cmap=plt.cm.Greys,)
plt.title("Confusion matrix")
plt.colorbar()
tick_marks = np.arange(len(label_names))
plt.xticks(tick_marks, label_names, rotation=90)
plt.yticks(tick_marks, label_names)
plt.tight_layout()
plt.ylabel('Actual Labels')
plt.xlabel('Predicted Labels')

In [ ]:
%matplotlib inline